In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [2]:
device_count = torch.cuda.device_count()
for i in range(device_count):
    print(torch.cuda.get_device_properties(i))

_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060 Laptop GPU', major=8, minor=6, total_memory=6143MB, multi_processor_count=30)


In [3]:
class FashionMNIST(Dataset):
    def __init__(self, train=True, transform=False):
        self.transform = None
        if transform: 
            self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))  
        ])
        self.path: str = r'/ml_notebooks/'
        
        self.data = datasets.FashionMNIST(
            root=self.path,
            train=train,
            download=True,
            transform= self.transform  
        )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx]  
        print('shape',np.array(img).shape,'\n')
        return img, label

In [4]:
train_data = FashionMNIST(train=True, transform=False)
len(train_data)

60000

In [5]:
train_data[0][0]

shape (28, 28) 



In [6]:
test_data = FashionMNIST(train=False, transform=False)
len(test_data)

10000

In [7]:
test_data[0][0]

shape (28, 28) 



In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.label_emb = nn.Embedding(10,10)
        self.model = nn.Sequential(
            nn.Linear(794,1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        ) 
    def forward(self, x, labels):
        x = x.view(x.size(0), 784) # 28*28 
        c = self.label_emb(labels)
        x = torch.cat([x,c], 1)
        out = self.model(x)
        return out.squeeze()
        

In [17]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.label_emb = nn.Embedding(10, 10) 
        self.model = nn.Sequential(
            nn.Linear(110, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, z, labels):
        z = z.view(z.size(0), 100)  
        c = self.label_emb(labels)  
        x = torch.cat([z, c], 1)  
        out = self.model(x)  
        return out.view(x.size(0), 28, 28)


In [19]:
gen = Generator().cuda()
disc = Discriminator().cuda()

In [20]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(disc.parameters(), lr=1e-4)
g_optimizer = torch.optim.Adam(gen.parameters(), lr=1e-4)

In [23]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    g_optimizer.zero_grad()
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, 10, batch_size))).cuda()
    fake_images = generator(z, fake_labels)
    validity = discriminator(fake_images, fake_labels)
    g_loss = criterion(validity, Variable(torch.ones(batch_size)).cuda())
    g_loss.backward()
    g_optimizer.step()
    return g_loss.data[0]

In [25]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    d_optimizer.zero_grad()

    # train with real images
    real_validity = discriminator(real_images, labels)
    real_loss = criterion(real_validity, Variable(torch.ones(batch_size)).cuda())
    
    # train with fake images
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, 10, batch_size))).cuda()
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels)
    fake_loss = criterion(fake_validity, Variable(torch.zeros(batch_size)).cuda())
    
    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss.data[0]

In [31]:
num_epochs = 30
n_critic = 5
display_step = 300
train_data = FashionMNIST(train=True, transform=True)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
for epoch in range(num_epochs):
    print('Starting epoch {}...'.format(epoch))
    for i, (images, labels) in enumerate(train_loader):
        real_images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        gen.train()
        batch_size = real_images.size(0)
        d_loss = discriminator_train_step(len(real_images), disc,
                                          gen, d_optimizer, criterion,
                                          real_images, labels)
        

        g_loss = generator_train_step(batch_size, disc, gen, g_optimizer, criterion)

    generator.eval()
    print('g_loss: {}, d_loss: {}'.format(g_loss, d_loss))
    z = Variable(torch.randn(9, 100)).cuda()
    labels = Variable(torch.LongTensor(np.arange(9))).cuda()
    sample_images = gen(z, labels).unsqueeze(1).data.cpu()
    grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
    plt.imshow(grid)
    plt.show()

Starting epoch 0...
shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 

shape (1, 28, 28) 



IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number